In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
from collections import Counter
from mmseg.apis import MMSegInferencer
from mmseg.apis import inference_model, init_model, show_result_pyplot
from PIL import Image

In [2]:
from mmseg.apis import MMSegInferencer
# models is a list of model names, and them will print automatically
models = MMSegInferencer.list_models('mmseg')

12/06 19:27:00 - mmengine - INFO - model_name: ann_r50-d8_4xb2-40k_cityscapes-512x1024
model_name: ann_r101-d8_4xb2-40k_cityscapes-512x1024
model_name: ann_r50-d8_4xb2-40k_cityscapes-769x769
model_name: ann_r101-d8_4xb2-40k_cityscapes-769x769
model_name: ann_r50-d8_4xb2-80k_cityscapes-512x1024
model_name: ann_r101-d8_4xb2-80k_cityscapes-512x1024
model_name: ann_r50-d8_4xb2-80k_cityscapes-769x769
model_name: ann_r101-d8_4xb2-80k_cityscapes-769x769
model_name: ann_r50-d8_4xb4-80k_ade20k-512x512
model_name: ann_r101-d8_4xb4-80k_ade20k-512x512
model_name: ann_r50-d8_4xb4-160k_ade20k-512x512
model_name: ann_r101-d8_4xb4-160k_ade20k-512x512
model_name: ann_r50-d8_4xb4-20k_voc12aug-512x512
model_name: ann_r101-d8_4xb4-20k_voc12aug-512x512
model_name: ann_r50-d8_4xb4-40k_voc12aug-512x512
model_name: ann_r101-d8_4xb4-40k_voc12aug-512x512
model_name: apcnet_r50-d8_4xb2-40k_cityscapes-512x1024
model_name: apcnet_r101-d8_4xb2-40k_cityscapes-512x1024
model_name: apcnet_r50-d8_4xb2-40k_cityscapes-76

## Helper functions

In [49]:
# calculate iou between y_true and y_pred
def iou(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    if np.sum(union) == 0:
        return 0
    else:
        return np.sum(intersection) / np.sum(union)

In [50]:
# make inference
def make_inference(config,checkpoint, img_path, label_path):
    model = init_model(config, checkpoint, device='cuda:0')
    result = inference_model(model, img_path)
    y_pred = result.pred_sem_seg.data.cpu().numpy()
    y_true = np.array(Image.open(label_path))
    iou_score = iou(y_true, y_pred)
    return iou_score

In [68]:
# resize image and label and save them
from skimage.transform import resize

def resize_image_label(img_path, label_path, save_img_path, save_label_path, size):
    img = Image.open(img_path)
    label = Image.open(label_path)
    img = img.resize(size)
    label = label.resize(size)
    img.save(save_img_path)
    label.save(save_label_path)

In [71]:
img_path = '/workspaces/YJ_661_final_project/data/img_for_inference_2d/'
label_path = '/workspaces/YJ_661_final_project/data/annotation_2d_binary/'
save_img_path = '/workspaces/YJ_661_final_project/data/img_for_inference_2d_256*256/'
save_label_path = '/workspaces/YJ_661_final_project/data/annotation_2d_binary_256*256/'
size = (256,256)

for i in range(110):
    img_path_ = img_path + 'img_' + str(i) + '.png'
    label_path_ = label_path + 'img_' + str(i) + '.png'
    save_img_path_ = save_img_path + 'img_' + str(i) + '.png'
    save_label_path_ = save_label_path + 'img_' + str(i) + '.png'
    resize_image_label(img_path_, label_path_, save_img_path_, save_label_path_, size)
    

## Try different pre-trained models

### unet-s5-d16_pspnet_4xb4-40k_hrf-256x256

In [72]:
config_path = '/workspaces/YJ_661_final_project/src/unet-s5-d16_pspnet_4xb4-40k_hrf-256x256.py'
checkpoint_path = '/workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth'
img_path = '/workspaces/YJ_661_final_project/data/img_for_inference/img_0.png'

In [73]:
from mmengine import Config
cfg = Config.fromfile(config_path)
cfg.model.decode_head.num_classes = 2
cfg.model.auxiliary_head.num_classes = 2

In [53]:
iou_scores = []
for i in range(110):
    img_path = '/workspaces/YJ_661_final_project/data/img_for_inference_2d/img_{}.png'.format(i)
    label_path = '/workspaces/YJ_661_final_project/data/annotation_2d_binary/img_{}.png'.format(i)
    iou_score = make_inference(cfg, checkpoint_path, img_path, label_path)
    iou_scores.append(iou_score)

/opt/conda/envs/openmmlab/lib/python3.10/site-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/opt/conda/envs/openmmlab/lib/python3.10/site-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/opt/conda/envs/openmmlab/lib/python3.10/site-packages/mmseg/models/losses/cross_entropy_loss.py:249: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_2020122

In [54]:
print('Average IoU score: {}'.format(np.nanmean(iou_scores)))

Average IoU score: 0.00042271960742426665


In [74]:
iou_scores = []
for i in range(110):
    img_path = '/workspaces/YJ_661_final_project/data/img_for_inference_2d_256*256/img_{}.png'.format(i)
    label_path = '/workspaces/YJ_661_final_project/data/annotation_2d_binary_256*256/img_{}.png'.format(i)
    iou_score = make_inference(cfg, checkpoint_path, img_path, label_path)
    iou_scores.append(iou_score)

/opt/conda/envs/openmmlab/lib/python3.10/site-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/opt/conda/envs/openmmlab/lib/python3.10/site-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/opt/conda/envs/openmmlab/lib/python3.10/site-packages/mmseg/models/losses/cross_entropy_loss.py:249: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_20201227_181818-fdb7e29b.pth
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_unet_s5-d16_256x256_40k_hrf_2020122

In [75]:
print('Average IoU score: {}'.format(np.nanmean(iou_scores)))

Average IoU score: 0.0022152529426350794


### pspnet_r50-d8_4xb2-40k_cityscapes-512x1024

In [86]:
img_path = '/workspaces/YJ_661_final_project/data/img_for_inference_2d/'
label_path = '/workspaces/YJ_661_final_project/data/annotation_2d_binary/'
save_img_path = '/workspaces/YJ_661_final_project/data/img_for_inference_2d_512*1024/'
save_label_path = '/workspaces/YJ_661_final_project/data/annotation_2d_binary_512*1024/'
size = (512, 1024)

for i in range(110):
    img_path_ = img_path + 'img_' + str(i) + '.png'
    label_path_ = label_path + 'img_' + str(i) + '.png'
    save_img_path_ = save_img_path + 'img_' + str(i) + '.png'
    save_label_path_ = save_label_path + 'img_' + str(i) + '.png'
    resize_image_label(img_path_, label_path_, save_img_path_, save_label_path_, size)

In [87]:
config_path = '/workspaces/YJ_661_final_project/src/pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py'
checkpoint_path = '/workspaces/YJ_661_final_project/src/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

In [88]:
cfg = Config.fromfile(config_path)
cfg.model.decode_head.num_classes = 2
cfg.model.auxiliary_head.num_classes = 2

In [60]:
iou_scores = []
for i in range(110):
    img_path = '/workspaces/YJ_661_final_project/data/img_for_inference_2d/img_{}.png'.format(i)
    label_path = '/workspaces/YJ_661_final_project/data/annotation_2d_binary/img_{}.png'.format(i)
    iou_score = make_inference(cfg, checkpoint_path, img_path, label_path)
    iou_scores.append(iou_score)

Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth
The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([19, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 512, 1, 1]).
size mismatch for decode_head.conv_seg.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for auxiliary_head.conv_seg.weight: copying a param with shape torch.Size([19, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 256, 1, 1]).
size mismatch for auxiliary_head.conv_seg.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([2]).
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_00

In [61]:
print('Average IoU score: {}'.format(np.mean(iou_scores)))

Average IoU score: 0.006486622317332341


In [89]:
iou_scores = []
for i in range(110):
    img_path = '/workspaces/YJ_661_final_project/data/img_for_inference_2d_512*1024/img_{}.png'.format(i)
    label_path = '/workspaces/YJ_661_final_project/data/annotation_2d_binary_512*1024/img_{}.png'.format(i)
    iou_score = make_inference(cfg, checkpoint_path, img_path, label_path)
    iou_scores.append(iou_score)

Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth
The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([19, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 512, 1, 1]).
size mismatch for decode_head.conv_seg.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for auxiliary_head.conv_seg.weight: copying a param with shape torch.Size([19, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 256, 1, 1]).
size mismatch for auxiliary_head.conv_seg.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([2]).
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_00

In [90]:
print('Average IoU score: {}'.format(np.mean(iou_scores)))

Average IoU score: 0.0052223804736203005


### mobilenet-v2-d8_fcn_4xb2-80k_cityscapes-512x1024



In [62]:
# !mim download mmsegmentation --config mobilenet-v2-d8_fcn_4xb2-80k_cityscapes-512x1024 --dest .

processing mobilenet-v2-d8_fcn_4xb2-80k_cityscapes-512x1024...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MiB 43.5 MB/s eta 0:00:0000:0100:01
Successfully downloaded mobilenet-v2-d8_fcn_4xb2-80k_cityscapes-512x1024-20230224_185436-13fef4ea.pth to /workspaces/YJ_661_final_project/src
Successfully dumped mobilenet-v2-d8_fcn_4xb2-80k_cityscapes-512x1024.py to /workspaces/YJ_661_final_project/src


In [91]:
config_path = '/workspaces/YJ_661_final_project/src/mobilenet-v2-d8_fcn_4xb2-80k_cityscapes-512x1024.py'
checkpoint_path = '/workspaces/YJ_661_final_project/src/mobilenet-v2-d8_fcn_4xb2-80k_cityscapes-512x1024-20230224_185436-13fef4ea.pth'

In [92]:
cfg = Config.fromfile(config_path)
cfg.model.decode_head.num_classes = 2
cfg.model.auxiliary_head.num_classes = 2

In [93]:
iou_scores = []
for i in range(110):
    img_path = '/workspaces/YJ_661_final_project/data/img_for_inference_2d_512*1024/img_{}.png'.format(i)
    label_path = '/workspaces/YJ_661_final_project/data/annotation_2d_binary_512*1024/img_{}.png'.format(i)
    iou_score = make_inference(cfg, checkpoint_path, img_path, label_path)
    iou_scores.append(iou_score)

Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/mobilenet-v2-d8_fcn_4xb2-80k_cityscapes-512x1024-20230224_185436-13fef4ea.pth
The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([19, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 512, 1, 1]).
size mismatch for decode_head.conv_seg.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for auxiliary_head.conv_seg.weight: copying a param with shape torch.Size([19, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 256, 1, 1]).
size mismatch for auxiliary_head.conv_seg.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([2]).
Loads checkpoint by local backend from path: /workspaces/YJ_661_final_project/src/mobilenet-v2-d8_fcn_4xb2-80k_cityscape

In [94]:
print('Average IoU score: {}'.format(np.mean(iou_scores)))

Average IoU score: 0.005859110988836194
